In [56]:
import keras
from keras import layers
import numpy as np
import random
import io

In [57]:
text = "Lys_mykyta.txt"

In [58]:
with io.open(text, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Corpus length: 86526
Total chars: 49


In [59]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

Number of sequences: 28829


In [60]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [61]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

In [62]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [63]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [64]:
model.fit(x, y, batch_size=128, epochs=25)

Epoch 1/25
226/226 [==============================] - 15s 54ms/step - loss: 2.7476
Epoch 2/25
226/226 [==============================] - 10s 43ms/step - loss: 2.3643
Epoch 3/25
226/226 [==============================] - 10s 43ms/step - loss: 2.2205
Epoch 4/25
226/226 [==============================] - 9s 42ms/step - loss: 2.1055
Epoch 5/25
226/226 [==============================] - 10s 42ms/step - loss: 2.0039
Epoch 6/25
226/226 [==============================] - 10s 44ms/step - loss: 1.9031
Epoch 7/25
226/226 [==============================] - 10s 43ms/step - loss: 1.8007
Epoch 8/25
226/226 [==============================] - 10s 43ms/step - loss: 1.6983
Epoch 9/25
226/226 [==============================] - 10s 43ms/step - loss: 1.6050
Epoch 10/25
226/226 [==============================] - 10s 42ms/step - loss: 1.5236
Epoch 11/25
226/226 [==============================] - 10s 42ms/step - loss: 1.4528
Epoch 12/25
226/226 [==============================] - 9s 42ms/step - loss: 1.3892
Epo

In [83]:
def generate_text(start_index, length):
        generated_text = ""
        sentence = text[start_index : start_index + maxlen]
        x_pred = np.zeros((1, maxlen, len(chars)))
        for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, 0.1)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated_text += next_char
        return generated_text

In [84]:
start_index = random.randint(0, len(text) - maxlen - 1)

In [85]:
print(f"Generated text: {generate_text(start_index, 100)}")

Generated text:  почав він, про повічиться, то він смаєть!  – друже! – а мурлика привдав крикнув лис микита голова с
